In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing required libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# **Importing the dataset**

In [ ]:
college = pd.read_csv('../input/forbes-americas-top-colleges-2019/ForbesAmericasTopColleges2019.csv')

In [ ]:
college.head()

In [ ]:
college.shape

In [ ]:
college.nunique()

# **Visualizations**

In [ ]:
sns.relplot(data=college, x="Rank", y="Undergraduate Population", hue="Acceptance Rate", col="Public/Private")

In [ ]:
sns.lmplot(data=college, x="Total Annual Cost", y="Undergraduate Population", col="Public/Private", hue = 'Public/Private')

In [ ]:
sns.stripplot(x='Public/Private', y='Alumni Salary', alpha = 0.5, data=college)

In [ ]:
sns.lineplot(data = college, x='Total Annual Cost', y = 'Average Grant Aid', hue = 'Public/Private')

# **Dropping unwanted columns**

In [ ]:
clg = college.drop(columns = ['Name','City','State','Website','Rank'])

In [ ]:
clg.head()

# **Checking for null values**

In [ ]:
clg.isnull().sum()

# **Filling null values using interpolation**

In [ ]:
clg['Net Price'] = clg['Net Price'].interpolate()
clg['Average Grant Aid'] = clg['Average Grant Aid'].interpolate()
clg['Alumni Salary'] = clg['Alumni Salary'].interpolate()
clg['Acceptance Rate'] = clg['Acceptance Rate'].interpolate()
clg['SAT Lower']= clg['SAT Lower'].interpolate()
clg['SAT Upper']=clg['SAT Upper'].interpolate()
clg['ACT Lower']=clg['ACT Lower'].interpolate()
clg['ACT Upper']=clg['ACT Upper'].interpolate()

In [ ]:
clg.isnull().sum()

# **Dropping 'Alumni Salary' cloumn as we will use it as our Dependent Variable**

In [ ]:
clg_1 = clg.drop(columns=['Alumni Salary'])

In [ ]:
clg_1.head()

# **Creating Independent Variable (x) and Dependent Variable (y)**

In [ ]:
x = clg_1.iloc[:,0:].values

In [ ]:
y = clg['Alumni Salary'].values

# **Handling the categoerical 'Public/Private' column**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0])], remainder = 'passthrough')

In [ ]:
x = np.array(ct.fit_transform(x))

In [ ]:
x

# **Splitting data into training and testing sets**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

# **Scaling x_train and x_train**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
x_train_1 = sc.fit_transform(x_train)

In [ ]:
x_test_1 = sc.transform(x_test)

# **Creating activation model**

In [ ]:
ann = tf.keras.models.Sequential()

# **1st Hidden layer**

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation = 'relu'))

# **2nd Hidden layer**

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation = 'relu'))

# **3rd Hidden layer**

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation = 'relu'))

# **Output layer**

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation = 'linear'))

# **Defining optimizer and loss function**

In [ ]:
ann.compile(optimizer = 'adam', loss = 'mse')

# **Training our data on the activation model**

In [ ]:
ann.fit(x_train_1, y_train, batch_size = 32, epochs = 10000)

# **Predicting our x_test based on model**

In [ ]:
y_pred=ann.predict(x_test_1)

In [ ]:
y_pred.shape

# **Concatinating y_test and y_pred**

In [ ]:
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

In [ ]:
y = y_test - y_pred

In [ ]:
y

# **Visualizations based on the prediction**

In [ ]:
plt.scatter(y_test, y_pred)

In [ ]:
plt.scatter(x_test[:,6],y_test)
plt.scatter(x_test[:,6],y_pred, color = 'red')
plt.xlabel('Total Annual Cost')
plt.ylabel('Alumni Salary')

In [ ]:
plt.scatter(x_test[:,3],y_test)
plt.scatter(x_test[:,3],y_pred, color = 'red')
plt.xlabel('Student Population')
plt.ylabel('Alumni Salary')